# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import random
from moviepy.editor import VideoFileClip
from IPython.display import display, HTML
%matplotlib inline

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images  
`cv2.cvtColor()` to grayscale or change color  
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [2]:
import math

x1_left_li = []
x2_left_li = []
x1_right_li = []
x2_right_li = []
x1_left_a = 0
x2_left_a = 0
x1_right_a = 0
x2_right_a = 0

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def mvg_avg_left(x1, x2, qty = 5):
    """
    Calculates a simple moving average
    `x1` is the x1 value
    `x2` is the x2 value
    `qty` is the number of values to consider in the moving average
    """
    global x1_left_li, x2_left_li, x1_left_a, x2_left_a
    if len(x1_left_li) > qty:
        x1_left_li.pop(0) 
        x2_left_li.pop(0)
    x1_left_li.append(x1)
    x2_left_li.append(x2)
    x1_left_a = int(sum(x1_left_li) / float(len(x1_left_li)))
    x2_left_a = int(sum(x2_left_li) / float(len(x2_left_li)))
    
    
def mvg_avg_right(x1, x2, qty = 5):
    """
    Calculates a simple moving average
    `x1` is the x1 value
    `x2` is the x2 value
    `qty` is the number of values to consider in the moving average
    """
    global x1_right_li, x2_right_li, x1_right_a, x2_right_a
    if len(x1_right_li) > qty: 
        x1_right_li.pop(0) 
        x2_right_li.pop(0) 
    x1_right_li.append(x1)
    x2_right_li.append(x2)
    x1_right_a = int(sum(x1_right_li) / float(len(x1_right_li)))
    x2_right_a = int(sum(x2_right_li) / float(len(x2_right_li)))

def draw_lines(img, lines, avg_qty = 5, y_min = 315, color=[255, 0, 0], thickness=10):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    # original solution
    # for line in lines:
    #     for x1,y1,x2,y2 in line:
    #         cv2.line(img, (x1, y1), (x2, y2), color, thickness)

    # new solution
      # Generate lists to store values
    x1_left = []
    x2_left = []
    y1_left = []
    y2_left = []
    x1_right = []
    x2_right = []
    y1_right = []
    y2_right = []
    
    # Add the line values to the appropriate list
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = (y2-y1)/(x2-x1) # get the slope
            if slope >= 0:
                # positive slope, right line
                x1_right.append(x1)
                x2_right.append(x2)
                y1_right.append(y1)
                y2_right.append(y2)
            else:
                # negative slope, left line
                x1_left.append(x1)
                x2_left.append(x2)
                y1_left.append(y1)
                y2_left.append(y2)
            
    # Compute the average value for each point
    x1_left_avg = (sum(x1_left) / float(len(x1_left))) if len(x1_left) != 0 else 0
    x2_left_avg = (sum(x2_left) / float(len(x2_left))) if len(x2_left) != 0 else 0
    y1_left_avg = (sum(y1_left) / float(len(y1_left))) if len(y1_left) != 0 else 0
    y2_left_avg = (sum(y2_left) / float(len(y2_left))) if len(y2_left) != 0 else 0
    x1_right_avg = (sum(x1_right) / float(len(x1_right))) if len(x1_right) != 0 else 0
    x2_right_avg = (sum(x2_right) / float(len(x2_right))) if len(x2_right) != 0 else 0
    y1_right_avg = (sum(y1_right) / float(len(y1_right))) if len(y1_right) != 0 else 0
    y2_right_avg = (sum(y2_right) / float(len(y2_right))) if len(y2_right) != 0 else 0
    
    # Get the slope denominator for each side
    left_denom = (x2_left_avg - x1_left_avg)
    right_denom = (x2_right_avg - x1_right_avg) 
    
    # Get the x,y dimension of the image
    x = img.shape[1]
    y = img.shape[0]
    
    # Calculate the extended lines and get the moving average
    if left_denom != 0:
        # Get the slope
        left_slope = ((y2_left_avg - y1_left_avg) / left_denom)
        
        # Get the intercept
        left_b = y2_left_avg - left_slope * x2_left_avg
        
        # Calculate lines
        x1 = int((y_min - left_b) / left_slope)
        x2 = int((y - left_b) / left_slope)
        
        # Update moving averages if difference < 20% from average
        mvg_avg_left(x1, x2, avg_qty)
        
    if right_denom != 0:
        # Get the slope
        right_slope = ((y2_right_avg - y1_right_avg) / right_denom)
        
        # Get the intercept
        right_b = y2_right_avg - right_slope * x2_right_avg
        
        # Calculate lines
        x1 = int((y_min - right_b) / right_slope)
        x2 = int((y - right_b) / right_slope)
        
        # Update moving averages
        mvg_avg_right(x1, x2, avg_qty)
    
    # Plot the lines
    cv2.line(img, (x1_left_a, y_min), (x2_left_a, y), color, thickness)
    cv2.line(img, (x1_right_a, y_min), (x2_right_a, y), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

def showImagesInHtml(images, dir):
    """
    Shows the list of `images` names on the directory `dir` as HTML embeded on the page.
    Author: darienmt
    """
    randomNumber = random.randint(1, 100000)
    buffer = "<div>"
    for img in images:
        imgSource = dir + img + "?" + str(randomNumber)
        buffer += """<img src="{0}" width="300" height="110" style="float:left; margin:1px"/>""".format(imgSource)
    buffer += "</div>"
    display(HTML(buffer))

def saveImages(images, outputDir, imageNames, isGray=0):
    """
    Writes the `images` to the `outputDir` directory using the `imagesNames`.
    It creates the output directory if it doesn't exists.
    Author: darienmt
    Example:
    
    saveImages([img1], 'tempDir', ['myImage.jpg'])
    
    Will save the image on the path: tempDir/myImage.jpg
    
    """
    if not os.path.exists(outputDir):
        os.makedirs(outputDir)
        
    zipped = list(map(lambda imgZip: (outputDir + imgZip[1], imgZip[0]), zip(images, imageNames)))
    for imgPair in zipped:
        if isGray:
            plt.imsave(imgPair[0], imgPair[1], cmap='gray')
        else :
            plt.imsave(imgPair[0], imgPair[1])
        
def doSaveAndDisplay(images, outputDir, imageNames, somethingToDo, isGray=0):
    """
    Applies the lambda `somethingToDo` to `images`, safe the results at the directory `outputDir`,
    and render the results in html.
    
    It returns the output images.
    Author: darienmt
    """
    outputImages = list(map(somethingToDo, images))
    saveImages(outputImages, outputDir, imageNames, isGray)
    showImagesInHtml(imageNames, outputDir)
    return outputImages    

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [9]:
# Step 1 - Load test images
import os
test_img_dir = 'test_images/'
test_img_names = os.listdir(test_img_dir)
test_images = list(map(lambda img: plt.imread(test_img_dir + img), test_img_names))
showImagesInHtml(test_img_names, test_img_dir)

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [10]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images_output directory.
# Update: quochung11@gmail.com

# Step 2 - convert the images to grayscale
test_images_grayscale = doSaveAndDisplay(test_images, 'test_images_01_grayscale/', test_img_names, grayscale, 1)

In [11]:
# Step 3 - Applying Gaussionblur filter for Images smoothing
blur_kernel_size = 15
blurAction = lambda img:gaussian_blur(img, blur_kernel_size)
test_images_gaussiansmoothing = doSaveAndDisplay(test_images_grayscale, 'test_images_02_Gaussiansmoothing/', test_img_names, blurAction, 1)

In [12]:
# Step 4 - Applying Canny transform for edges detection
canny_low_threshold = 30
canny_high_threshold = 150
cannyAction = lambda img:canny(img, canny_low_threshold, canny_high_threshold)
test_images_canny = doSaveAndDisplay(test_images_gaussiansmoothing, 'test_images_03_canny_transform/', test_img_names, cannyAction)

In [13]:
# Step 5 - Applying ROI 
def ROIAction(img):
    ysize = img.shape[0]
    xsize = img.shape[1]
    reg_of_interest = np.array([ [0, ysize], [xsize/2,(ysize/2)+60], [xsize,ysize] ], np.int32)
    return region_of_interest(img, [reg_of_interest])

test_images_roi = doSaveAndDisplay(test_images_canny, 'test_images_04_roi/', test_img_names, ROIAction)

In [14]:
# Step 6 - Applying Hough transform for lines segment
rho = 2                     # distance resolution in pixels of the Hough grid
theta = np.pi/180           # angular resolution in radians of the Hough grid
threshold = 20              # minimum number of votes (intersections in Hough grid cell)
min_line_length = 60       # minimum number of pixels making up a line
max_line_gap = 5            # maximum gap in pixels between connectable line segments

houghAction = lambda img: hough_lines(img, rho, theta, threshold, min_line_length, max_line_gap)

test_images_hough = doSaveAndDisplay(test_images_roi, 'test_images_05_houghtransform/', test_img_names, houghAction)

In [15]:
# Step 7 - Merged frame
test_image_merge_temp = list(map(lambda imgs: weighted_img(imgs[0], imgs[1]), zip(test_images,test_images_hough)))
test_images_merge = doSaveAndDisplay(test_image_merge_temp, 'test_images_06_merge/', test_img_names, lambda img: img)

In [ ]:
## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [4]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [5]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)

    # define params
    blur_kernel_size        = 15
    canny_low_threshold     = 30
    canny_high_threshold    = 150
    rho             = 2                     
    theta           = np.pi/180           
    threshold       = 20              
    min_line_length = 60       
    max_line_gap    = 5

    # run image processing
    vid_fr_gray = grayscale(image)
    vid_fr_blur = gaussian_blur(vid_fr_gray, blur_kernel_size)
    vid_fr_cann = canny(vid_fr_blur, canny_low_threshold, canny_high_threshold)
    vid_fr_roi  = ROIAction(vid_fr_cann)
    vid_fr_houg = hough_lines(vid_fr_roi, rho, theta, threshold, min_line_length, max_line_gap)
    vid_fr_merg = weighted_img(vid_fr_houg, image)
    
    # return result
    return vid_fr_merg

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [18]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)


t:   1%|          | 3/251 [01:17<1:46:29, 25.76s/it, now=None]

t:   2%|▏         | 3/125 [00:28<19:31,  9.61s/it, now=None]

t:   0%|          | 0/681 [00:00<?, ?it/s, now=None]

t:   3%|▎         | 19/681 [00:00<00:03, 188.94it/s, now=None]Moviepy - Building video test_videos_output/solidYellowLeft.mp4.
Moviepy - Writing video test_videos_output/solidYellowLeft.mp4



t:   6%|▌         | 40/681 [00:00<00:03, 194.72it/s, now=None]

t:   7%|▋         | 51/681 [00:00<00:04, 147.13it/s, now=None]

t:  10%|▉         | 65/681 [00:00<00:04, 141.81it/s, now=None]

t:  11%|█         | 76/681 [00:00<00:04, 128.63it/s, now=None]

t:  13%|█▎        | 88/681 [00:00<00:04, 123.53it/s, now=None]

t:  15%|█▍        | 100/681 [00:00<00:04, 119.20it/s, now=None]

t:  16%|█▋        | 111/681 [00:00<00:05, 111.60it/s, now=None]

t:  18%|█▊        | 122/681 [00:00<00:05, 107.59it/s, now=None]

t:  20%|█▉        | 134/681 [00:01<00:05, 108.56it/s, now=None]

t:  21%|██▏       | 145/681 [00:01<00:05, 105.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [19]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)


t:   1%|          | 3/251 [01:28<2:02:19, 29.60s/it, now=None]

t:   2%|▏         | 3/125 [00:40<27:19, 13.44s/it, now=None]

t:   0%|          | 0/125 [00:00<?, ?it/s, now=None]Moviepy - Building video test_videos_output/challenge.mp4.
Moviepy - Writing video test_videos_output/challenge.mp4



OverflowError: cannot convert float infinity to integer

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))